In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import OneHotEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
from imblearn.over_sampling import SMOTE
import matplotlib.pyplot as plt
import seaborn as sns
import scipy

df = pd.read_csv("prepared_jobs_dataset.csv")
df.head()


,JobPosting_Key,job_title_short,CountryName,company_name,Skill_Name,skill_text,remote_flag
0,2,data analyst,mexico,hewlett packard enterprise,"['r', 'nosql', 'sql', 'python', 'excel', 'powe...",r nosql sql python excel power bi sap tableau,0
1,3,data engineer,germany,alpha augmented services,"['c#', 'sql', 'azure', 'docker', 'python', 'ai...",c# sql azure docker python airflow dax jenkins...,0
2,4,data engineer,united states,southwest research institute,"['aws', 'matlab', 'tensorflow', 'python', 'jav...",aws matlab tensorflow python java pytorch c++ ...,0
3,5,data engineer,sudan,kristina daniel,"['aws', 'gitlab', 'bash', 'puppet', 'python', ...",aws gitlab bash puppet python jenkins git ansi...,0
4,6,data engineer,united states,smart folks inc,"['gcp', 'sql', 'python']",gcp sql python,1


In [3]:
from sklearn.feature_extraction.text import TfidfVectorizer
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import OneHotEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
from imblearn.over_sampling import SMOTE
import matplotlib.pyplot as plt
import seaborn as sns
import scipy
df_remote = df[df['remote_flag'] == 1]  # minority class
df_onsite = df[df['remote_flag'] == 0]  # majority class

df_onsite_sampled = df_onsite.sample(n=len(df_remote), random_state=42)

df_balanced = pd.concat([df_remote, df_onsite_sampled]).sample(frac=1, random_state=42)

# Now df_balanced is used for feature extraction and modeling
df = df_balanced
tfidf_skills = TfidfVectorizer(max_features=500)
X_skills = tfidf_skills.fit_transform(df['skill_text'])
tfidf_title = TfidfVectorizer(max_features=200)
X_title = tfidf_title.fit_transform(df['job_title_short'])
tfidf = TfidfVectorizer(max_features=500)
X_tfidf = tfidf.fit_transform(df['skill_text'])
from sklearn.preprocessing import OneHotEncoder
import scipy

# One-hot encoding for CountryName
ohe = OneHotEncoder(sparse_output=False, handle_unknown='ignore')
X_country = ohe.fit_transform(df[['CountryName']])

# Combine TF-IDF and one-hot features
X_combined = scipy.sparse.hstack([X_skills, X_title, scipy.sparse.csr_matrix(X_country)])
X_combined_dense = X_combined.toarray()

In [4]:
y = df['remote_flag']


In [5]:
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import OneHotEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
from imblearn.over_sampling import SMOTE
import matplotlib.pyplot as plt
import seaborn as sns
import scipy
X_train, X_test, y_train, y_test = train_test_split(
    X_combined_dense, y, test_size=0.2, random_state=42, stratify=y
)

# Apply SMOTE on dense arrays
from imblearn.over_sampling import SMOTE
smote = SMOTE(random_state=42)
X_train_res, y_train_res = smote.fit_resample(X_train, y_train)


In [6]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import OneHotEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
from imblearn.over_sampling import SMOTE
import matplotlib.pyplot as plt
import seaborn as sns
import scipy
rf = RandomForestClassifier(
    n_estimators=200,
    max_depth=12,
    min_samples_split=5,
    min_samples_leaf=3,
    class_weight='balanced',
    random_state=42,
    n_jobs=-1
)
rf.fit(X_train_res, y_train_res)

# Save the model and preprocessors
import joblib
joblib.dump(rf, 'rf_remote_model.joblib')
joblib.dump(tfidf_skills, 'tfidf_skills.joblib')
joblib.dump(tfidf_title, 'tfidf_title.joblib')
joblib.dump(ohe, 'ohe_rf.joblib')

# =====  Predictions =====
y_pred = rf.predict(X_test)

# =====  Evaluation =====
print("Accuracy:", accuracy_score(y_test, y_pred))
print("\nClassification Report:\n", classification_report(y_test, y_pred))

# Confusion matrix







plt.show()plt.title("Confusion Matrix")plt.ylabel("Actual")plt.xlabel("Predicted")sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=[0,1], yticklabels=[0,1])plt.figure(figsize=(6,5))cm = confusion_matrix(y_test, y_pred)cm = confusion_matrix(y_test, y_pred)
plt.figure(figsize=(6,5))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=[0,1], yticklabels=[0,1])
plt.xlabel("Predicted")
plt.ylabel("Actual")
plt.title("Confusion Matrix")
plt.show()

SyntaxError: invalid syntax (831334885.py, line 45)